In [103]:
import numpy as np
from astropy.io import fits 
import matplotlib.pyplot as plt
from astropy.table import Table, Column
from matplotlib import rc
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 8,
    "axes.titlesize": 8
})
cm = 1/2.54

In [104]:
cat = Table.read("aegean_testing/white_testingarea_default_comp.fits")
cat.colnames

['island',
 'source',
 'background',
 'local_rms',
 'ra_str',
 'dec_str',
 'ra',
 'err_ra',
 'dec',
 'err_dec',
 'peak_flux',
 'err_peak_flux',
 'int_flux',
 'err_int_flux',
 'a',
 'err_a',
 'b',
 'err_b',
 'pa',
 'err_pa',
 'flags',
 'residual_mean',
 'residual_std',
 'uuid',
 'psf_a',
 'psf_b',
 'psf_pa']

In [109]:
good_inds = np.where(cat['err_a']!=-1)

mean_erra = np.mean(cat['err_a'][good_inds])
mean_a = np.mean(cat['a'][good_inds])

mean_errb = np.mean(cat['err_b'][good_inds])
mean_b = np.mean(cat['b'][good_inds])

frac_pa_err = np.std((cat['err_pa'][good_inds]/cat['pa'][good_inds]))
frac_a_err = np.std((cat['err_a'][good_inds]/cat['a'][good_inds]))
frac_b_err = np.std((cat['err_b'][good_inds]/cat['b'][good_inds]))
# mean_erra = np.mean(cat['err_a'][good_inds])
# mean_a = np.mean(cat['a'][good_inds])
# std_b = np.mean(cat['err_b'][good_inds])
# std_pa = np.mean(cat['err_pa'][good_inds])
print(frac_a_err)


0.02989882


In [97]:
cat["err_ra"][np.where(cat["err_ra"]==-1)]=0.771
cat["err_dec"][np.where(cat["err_dec"]==-1)]=0.806
# cat["err_a"][np.where(cat["err_a"]==-1)]=std_a
# cat["err_b"][np.where(cat["err_b"]==-1)]=std_b
# cat["err_pa"][np.where(cat["err_pa"]==-1)]=std_pa
bad_inds = np.where(cat['err_a'] == -1)
bad_peak_inds = np.where(cat['err_peak_flux']==-1)
bad_int_inds = np.where(cat["err_int_flux"]==-1)
# bad_flags = np.where(cat["flags"]!= 0)

In [98]:
for i in bad_inds:
    cat['err_a'][i] = frac_a_err*cat['a'][i]
    cat['err_b'][i] = frac_b_err*cat['b'][i]
    cat['err_pa'][i] = frac_pa_err*cat['pa'][i]

for i in bad_peak_inds:
    cat["err_peak_flux"][i] = np.sqrt(cat['local_rms'][i] ** 2 + (cat["peak_flux"][i] * 0.02)**2)

for i in bad_int_inds:
    cat["err_int_flux"][i] = np.sqrt((cat['err_a'][i]/cat['a'][i])**2+(cat['err_b'][i]/cat['b'][i])**2+(cat['err_peak_flux'][i]/cat['peak_flux'][i])**2)*cat['int_flux'][i]



In [99]:
vbad_inds = np.where(np.isnan(cat['err_ra']))
print(vbad_inds)

for i in vbad_inds:
    cat["err_peak_flux"][i] = np.sqrt(cat['local_rms'][i] ** 2 + (cat["peak_flux"][i] * 0.02)**2)
    cat["err_int_flux"][i] = np.sqrt((cat['err_a'][i]/cat['a'][i])**2+(cat['err_b'][i]/cat['b'][i])**2+(cat['err_peak_flux'][i]/cat['peak_flux'][i])**2)*cat['int_flux'][i]
    cat["err_ra"][i]=0.771
    cat["err_dec"][i]=0.806

(array([1221, 1255, 3917]),)


In [102]:
cat["flags"][np.where(cat['flags']!=0)] = 0
cat.write('aegean_testing/white_nice_noflags.fits', format='fits', overwrite=True)  